In [ ]:
from grid.refine import *

df = observed_rates()

In [ ]:
df.query('width > 64')

In [ ]:
#df = data.load()

interp = df.interpolate().where(df.bfill().notnull())
stacked = interp.stack([1, 2, 3]).reset_index().dropna()
stacked['flops'] = flops(stacked)
stacked = stacked[stacked.elo_std < .5]

best = stacked.groupby(['_time', 'boardsize']).elo.idxmax()
best = stacked.loc[best]

index = 10**np.linspace(0, np.log10(stacked.flops.max()), 1001)
regular = pd.pivot_table(stacked, 'elo', ('flops',), ('boardsize', 'width', 'depth')).ffill().reindex(index, method='nearest')
frontier = regular.expanding().max().groupby(axis=1, level=0).max().div(data.min_elos(), axis=1).mul(-1).add(1)

with plt.style.context('seaborn-poster'):
    slope = (frontier > .95).drop(11, axis=1).idxmax()
    ax = slope.plot(logy=True, marker='o', linestyle='--', grid=True)
    ax.set_ylabel('training FLOPS')
    ax.set_xlabel('boardsize')
    ax.set_title('FLOPS needed to hit 95% of perfect play is roughly linear')

In [ ]:
smoothed = frontier.rolling(10, 5, center=True).mean().drop(11, axis=1)
sixes = (smoothed > .6).idxmax()
nines = (smoothed > .95).idxmax()


In [ ]:
((.95 - .6)*-data.min_elos())/(np.log10(nines) - np.log10(sixes))

In [ ]:
frontier = regular.expanding().max().groupby(axis=1, level=0).max()

In [ ]:
nines/sixes

In [ ]:
y = frontier.stack().rename('elo').reset_index()
y['shifted'] = y.flops.div(x.set_index('boardsize').flops[y.boardsize.values].values)

(ggplot(data=y)
    + geom_line(aes(x='shifted', y='elo', color='boardsize', group='boardsize'), size=2)
    + scale_x_continuous(trans='log10')
    + coord_cartesian((-4, 1))
    + labs(
        x='FLOPS as a fraction of 95% of perfect play',
        title='slope of the frontier seems the same across boardsizes?')
    + plots.mpl_theme())

In [ ]:
*slope =

In [ ]:
x.flops

In [ ]:
interp = df.interpolate().where(df.bfill().notnull()).ewm(span=5).mean()
stacked = interp.stack([1, 2, 3]).reset_index().dropna()
stacked['flops'] = flops(stacked)
stacked = stacked[stacked.elo_std < .5]

stacked['elo_flip'] = stacked.elo - data.min_elos()[stacked['boardsize'].values].values
stacked['g'] = stacked.boardsize.astype(str) + stacked.width.astype(str) + stacked.depth.astype(str)
finals = stacked.groupby(['boardsize', 'width', 'depth', '_time']).last().reset_index()

(ggplot(data=finals) + 
    geom_line(aes(x='flops', y='elo_flip', color='boardsize', group='g')) +
    scale_x_continuous(trans='log10'))
 

In [ ]:
smf.ols('np.log10(flops) ~ boardsize + 1', x).fit().summary2()

In [ ]:
x